# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [4]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [2]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

In [3]:
# Question 1: 10 countries with the lowest infant mortality rates

imr = pd.DataFrame(columns = ['country', 'infant_mortality_rate']) # create dataframe

for country in root.iter('country'):
    country_name = country.find('name').text # collect country names
    for inf_mort in country.findall('infant_mortality'):
        inf_mort = inf_mort.text # collect infant mortality rates
        imr = imr.append( # append data to dataframe
            { 'country': country_name,
              'infant_mortality_rate': inf_mort }, 
            ignore_index = True
        )
imr['infant_mortality_rate'] = pd.to_numeric(imr['infant_mortality_rate']) # convert infant mortality rates to numeric type
imr.sort_values(by = 'infant_mortality_rate').head(10) # solution


,country,infant_mortality_rate
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [21]:
# Question 2: 10 cities with the largest population

city_pop = pd.DataFrame(columns = ['city', 'year', 'population']) # create dataframe

for country in root.iter('country'):
    for city in country.iter('city'):
        city_name = city.find('name').text # collect city names
        
        for pop in city.findall('population'):
            year = pop.attrib['year'] # collect years associated with population counts
            population = pop.text # collect population counts
            
            if population != '0': # ignore population counts of zero
                
                city_pop = city_pop.append( # append data to dataframe
                    { 'city': city_name,
                      'year': year,
                      'population': population }, 
                    ignore_index = True
                )
            
city_pop[['year', 'population']] = city_pop[['year', 'population']].apply(pd.to_numeric) # convert years and population counts to numeric type 
city_pop = city_pop.drop_duplicates(subset = 'city', keep = 'last') # drop old population counts
city_pop.sort_values(by = 'population', ascending = False).head(10) # solution

,city,year,population
3747,Shanghai,2010,22315474
2604,Istanbul,2012,13710512
4300,Mumbai,2011,12442373
1543,Moskva,2013,11979529
3743,Beijing,2010,11716620
8204,São Paulo,2010,11152344
3751,Tianjin,2010,11090314
3361,Guangzhou,2010,11071424
4396,Delhi,2011,11034555
3368,Shenzhen,2010,10358381


In [6]:
# Question 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

eg = pd.DataFrame(columns = ['country', 'population', 'year', 'ethnic_group', 'percentage']) # create dataframe

for country in root.iter('country'):
    country_name = country.find('name').text # collect country names
    
    for pop in country.findall('population'):
        population = pop.text # collect population counts
        year = pop.attrib['year'] # collect years associated with population counts    
        
    for ethnic in country.iter('ethnicgroup'):
        ethnic_group = ethnic.text # collect ethnic group names
        percentage = ethnic.attrib['percentage'] # collect population percentages of ethnic groups
        
        eg = eg.append( # append data to dataframe
                { 'country': country_name,
                  'population': population,
                  'year': year,
                  'ethnic_group': ethnic_group,
                  'percentage': percentage },                
                ignore_index = True
        )  
            
eg

,country,population,year,ethnic_group,percentage
0,Albania,2800138,2011,Albanian,95
1,Albania,2800138,2011,Greek,3
2,Greece,10816286,2011,Greek,93
3,Macedonia,2059794,2011,Macedonian,64.2
4,Macedonia,2059794,2011,Albanian,25.2
5,Macedonia,2059794,2011,Turkish,3.9
6,Macedonia,2059794,2011,Gypsy,2.7
7,Macedonia,2059794,2011,Serb,1.8
8,Serbia,7120666,2011,Serb,82.9
9,Serbia,7120666,2011,Montenegrin,0.9


In [7]:
eg[['population', 'year', 'percentage']] = eg[['population', 'year', 'percentage']].apply(pd.to_numeric) # convert population counts, years, and percentages to numeric type
eg['ethnic_group_pop'] = eg['percentage'] / 100 * eg['population'] # create ethnic group population counts
eg.sort_values(by = ['ethnic_group', 'ethnic_group_pop']) # sort

,country,population,year,ethnic_group,percentage,ethnic_group_pop
609,Uganda,34856813,2014,Acholi,4.0,1.394273e+06
563,Djibouti,834036,2010,Afar,35.0,2.919126e+05
579,Ethiopia,84320987,2012,Afar,1.7,1.433457e+06
24,Andorra,78115,2011,African,5.0,3.905750e+03
408,Turks and Caicos Islands,31458,2012,African,90.0,2.831220e+04
156,Ireland,4588252,2011,African,1.1,5.047077e+04
402,Saint Kitts and Nevis,52352,2010,African,100.0,5.235200e+04
480,Suriname,541638,2012,African,10.0,5.416380e+04
384,Grenada,103328,2011,African,100.0,1.033280e+05
469,Uruguay,3286314,2011,African,4.0,1.314526e+05


In [19]:
eg_final = eg.copy() # create copy of dataframe
eg_final.drop(eg_final.columns[[0, 1, 2, 4]], axis = 1, inplace=True) # drop extraneous columns
eg_final.groupby('ethnic_group').sum().sort_values(by = 'ethnic_group_pop', ascending = False).head(10) # solution


,ethnic_group_pop
ethnic_group,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


In [22]:
# Question 4(a): Name and country of longest river

rivers = pd.DataFrame(columns = ['river', 'length', 'countries']) # create dataframe

for river in root.iter('river'):
    river_name = river.find('name').text # collect river names
    
    for item in river.iter('length'):
        river_length = item.text # collect river lengths

    for item in river.findall('located'):
        country_code = item.attrib['country'] # collect country codes       
        
        rivers = rivers.append( # append data to dataframe
            { 'river': river_name,
              'length': river_length,
              'countries': country_code },                
            ignore_index = True
        )  

rivers['length'] = pd.to_numeric(rivers['length']) # convert river lengths to numeric type
rivers = rivers.sort_values(by = 'length', ascending = False) # sort
rivers.head(3) # solution

,river,length,countries
246,Amazonas,6448.0,CO
247,Amazonas,6448.0,BR
248,Amazonas,6448.0,PE


In [15]:
# Question 4(b): Name and country of largest lake

lakes = pd.DataFrame(columns = ['lake', 'area', 'countries']) # create dataframe

for lake in root.iter('lake'):
    lake_name = lake.find('name').text # collect lake names
    
    for item in lake.iter('area'):
        lake_area = item.text # collect lake areas

    for item in lake.findall('located'):
        country_code = item.attrib['country'] # collect country codes       
        
        lakes = lakes.append( # append data to dataframe
            { 'lake': lake_name,
              'area': lake_area,
              'countries': country_code },                
            ignore_index = True
        )  

lakes['area'] = pd.to_numeric(lakes['area']) # convert lake areas to numeric type
lakes = lakes.sort_values(by = 'area', ascending = False) # sort
lakes.head(4) # solution


,lake,area,countries
59,Caspian Sea,386400.0,R
62,Caspian Sea,386400.0,TM
60,Caspian Sea,386400.0,KAZ
61,Caspian Sea,386400.0,IR


In [12]:
# Question 4(c): Name and country of airport at highest elevation

airports = pd.DataFrame(columns = ['airport', 'elevation', 'country']) # create dataframe

for airport in root.findall('airport'):
    airport_name = airport.find('name').text # collect airport names
    elevation = airport.find('elevation').text # collect elevations
    country_name = airport.attrib['country'] # collect country names
        
    airports = airports.append( # append data to dataframe
            { 'airport': airport_name,
              'elevation': elevation,
              'country': country_name },                
            ignore_index = True
    )  
airports['elevation'] = pd.to_numeric(airports['elevation']) # convert elevations to numeric type
airports = airports.sort_values(by = 'elevation', ascending = False) # sort
airports.head(1) # solution

,airport,elevation,country
80,El Alto Intl,4063.0,BOL
